In [ ]:
# Install required packages if missing (run this cell once)
import subprocess
import sys

def _ensure_pkg(*names):
    for name in names:
        try:
            __import__(name)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", name, "-q"])
_ensure_pkg("numpy", "pandas", "matplotlib", "seaborn", "scikit-learn", "xgboost", "catboost")

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Modelling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'numpy'

In [3]:
# Load the dataset (find stud.csv in notebook folder or workspace)
import os
_candidates = [
    'stud.csv',
    os.path.join(os.getcwd(), 'data', 'stud.csv'),
    os.path.join(os.getcwd(), 'notebook', 'data', 'stud.csv'),
    os.path.join(os.getcwd(), 'machine-learning', 'notebook', 'data', 'stud.csv'),
]
_csv_path = next((p for p in _candidates if os.path.isfile(p)), 'stud.csv')
df = pd.read_csv(_csv_path)

# Clean column names (remove spaces and special characters)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('/', '_')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
df.head()


NameError: name 'pd' is not defined

In [4]:
# Display basic info
print("Dataset Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())

: 

#### Preparing X and Y variables

In [ ]:
X = df.drop(columns=['math_score'])
X.head()

In [5]:
print("Categories in 'gender' variable:     ", end=" ")
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ", end=" ")
print(df['race_ethnicity'].unique())

print("Categories in 'parental level of education' variable:", end=" ")
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ", end=" ")
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ", end=" ")
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      

: 

In [ ]:
y = df['math_score']
y

In [ ]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude=["object", "string"]).columns
cat_features = X.select_dtypes(include=["object", "string"]).columns

print(f"Numerical features: {num_features.tolist()}")
print(f"Categorical features: {cat_features.tolist()}")

numeric_transformer = StandardScaler()
# Handle OneHotEncoder compatibility for different sklearn versions
try:
    # For sklearn >= 1.2
    oh_transformer = OneHotEncoder(sparse_output=False)
except TypeError:
    # For sklearn < 1.2
    try:
        oh_transformer = OneHotEncoder(sparse=False)
    except TypeError:
        oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),        
    ]
)
X = preprocessor.fit_transform(X)

# Convert to numpy array if needed (handles sparse matrices)
if hasattr(X, 'toarray'):
    X = X.toarray()

print(f"\nTransformed X shape: {X.shape}")

In [ ]:
# separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

#### Create an Evaluate Function to give all metrics after model Training

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)  # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

### Results

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"], ascending=False)

## Linear Regression

In [ ]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred) * 100
print(" Accuracy of the model is %.2f" % score)

## Plot y_pred and y_test

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

In [ ]:
sns.regplot(x=y_test, y=y_pred, ci=None, color='red')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Regression Plot: Actual vs Predicted')
plt.show()

#### Difference between Actual and Predicted Values

In [6]:
pred_df = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': y_pred, 'Difference': y_test - y_pred})
pred_df

: 